In [ ]:
!pip install mxnet-cu101mkl
!pip install gluonnlp pandas tqdm
!pip install sentencepiece

In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-4f2iwiz1
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-4f2iwiz1
  ERROR: Error [Errno 2] No such file or directory: 'git' while executing command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-4f2iwiz1
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?


In [30]:
import pandas as pd
import numpy as np
from mxnet.gluon import nn, rnn
from mxnet import gluon, autograd
import gluonnlp as nlp
from mxnet import nd 
import mxnet as mx
import time
import itertools
import random

from kobert.mxnet_kobert import get_mxnet_kobert_model
from kobert.utils import get_tokenizer

### Loading KoBERT

In [ ]:
sudo -H pip install mxnet --upgrade

In [31]:
# gpu 활성화 방법 ???
# ctx = mx.gpu()
ctx = mx.cpu()

In [32]:
# 모델 로딩
bert_base, vocab = get_mxnet_kobert_model(use_decoder=False, use_classifier=False, ctx=ctx)

using cached model
using cached model


In [33]:
# 토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [34]:
# Simple Dataset wrapper for lists and arrays.
ds = gluon.data.SimpleDataset([['나 보기가 역겨워', '김소월']])


"""버트는 입력 문장에 대해서 아래와 같은 작업을 요구
 1. 각 토큰의 Vocabulary 인덱스를 추출해 이를 정해진 길이의 벡터로 생성.
 2. 두 문장 혹은 하나의 문장이 들어올 수 있기 때문에 이들을 구분하기 위한 토큰 타입 벡터 생성
 3. 유효 길이 벡터
BERTSentenceTransform 은 위 작업을 편하게 진행해주는 함수"""
# 출처: http://freesearch.pe.kr/archives/4963
trans = nlp.data.BERTSentenceTransform(tok, max_seq_length=10)

list(ds.transform(trans))

[(array([   2, 1370, 2362, 5330, 3322,    3, 1316, 6607, 7028,    3],
        dtype=int32),
  array(10, dtype=int32),
  array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1], dtype=int32))]

### Loading Data

In [35]:
!wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
!wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

--2021-08-24 12:03:29--  https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.84.18, 2620:100:6034:18::a27d:5412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.84.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/374ftkec978br3d/ratings_train.txt [following]
--2021-08-24 12:03:30--  https://www.dropbox.com/s/dl/374ftkec978br3d/ratings_train.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc929c8f306b23331d335bbac2c5.dl.dropboxusercontent.com/cd/0/get/BU2GI8c-X5xgTfFu-aThzw1jCZqwSCP5F_FHs8uSt0KQeVh68Z0CJ-cLqAMqBDSH6Z-WusBBEcD1tmwSmihCvFWss1INzHG36npXZLj86bif3lX08uNzTISSA5l2Cg5i3UmpsRFND79ocmwQplHRyZZ_/file?dl=1# [following]
--2021-08-24 12:03:30--  https://uc929c8f306b23331d335bbac2c5.dl.dropboxusercontent.com/cd/0/get/BU2GI8c-X5xgTfFu-aThzw1jCZqwSCP5F_FHs8uSt0KQeVh68Z0CJ-cLqAMqBDSH6Z-Wus

In [36]:
"""Common tab separated text dataset that reads text fields based on provided sample splitter
    and field separator.

    The returned dataset includes samples, each of which can either be a list of text fields
    if field_separator is specified, or otherwise a single string segment produced by the
    sample_splitter."""
# 출처: https://nlp.gluon.ai/_modules/gluonnlp/data/dataset.html

dataset_train = nlp.data.TSVDataset("ratings_train.txt?dl=1", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("ratings_test.txt?dl=1", field_indices=[1,2], num_discard_samples=1)

In [37]:
# 모든 학습 데이터는 배치단위로 입/출력이 정의되기 때문에 입력 데이터 처리와 
# 레이블 처리를 동시에 배치 출력하기 위해 Dataset 클래스를 정의
# 출처: http://freesearch.pe.kr/archives/4963

class BERTDataset(mx.gluon.data.Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        sent_dataset = gluon.data.SimpleDataset([[
            i[sent_idx],
        ] for i in dataset])
        self.sentences = sent_dataset.transform(transform)
        self.labels = gluon.data.SimpleDataset(
            [np.array(np.int32(i[label_idx])) for i in dataset])

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [38]:
max_len = 128

In [39]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [40]:
# 모든 버트 모델은 첫 클래스 레이블을 pooler 라고 네이밍 하고 있고 이 pooler 결과를 받아 
# FC(fully connected) 레이어 하나를 추가해 간단하게 구성
# 출처: http://freesearch.pe.kr/archives/4963

class BERTClassifier(nn.Block):
    def __init__(self,
                 bert,
                 num_classes=2,
                 dropout=None,
                 prefix=None,
                 params=None):
        super(BERTClassifier, self).__init__(prefix=prefix, params=params)
        self.bert = bert
        with self.name_scope():
            self.classifier = nn.HybridSequential(prefix=prefix)
            if dropout:
                self.classifier.add(nn.Dropout(rate=dropout))
            self.classifier.add(nn.Dense(units=num_classes))

    def forward(self, inputs, token_types, valid_length=None):
        _, pooler = self.bert(inputs, token_types, valid_length)
        return self.classifier(pooler)
                                           

In [41]:
 # 파라미터 설명
# bert: BERTModel
#    Bidirectional encoder with transformer.
# num_classes : int, default is 2
#    The number of target classes.
# dropout : float or None, default 0.0.
#    Dropout probability for the bert output.
# 출처: https://nlp.gluon.ai/_modules/gluonnlp/model/bert.html
model = BERTClassifier(bert_base, num_classes=2, dropout=0.1)

# 분류 레이어만 초기화 한다. 
# Create a initializer that initialize the weight with normal(0, sd)
# 출처: https://mxnet.apache.org/versions/1.6/api/r/docs/api/mx.init.normal.html
model.classifier.initialize(init=mx.init.Normal(0.02), ctx=ctx)
model.hybridize()

# softmax cross entropy loss for classification
# SoftmaxCELoss(): Computes the softmax cross entropy loss
loss_function = gluon.loss.SoftmaxCELoss()

metric = mx.metric.Accuracy()

In [42]:
batch_size = 32
lr = 5e-5

# DataLoader - Loads data from a dataset and returns mini-batches of data.
# num_workers (int, default 0) – The number of multiprocessing workers to use for data preprocessing.
# 출처: http://34.201.8.176/versions/io/api/python/gluon/data.html#mxnet.gluon.data.DataLoader
train_dataloader = mx.gluon.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = mx.gluon.data.DataLoader(data_test, batch_size=int(batch_size/2), num_workers=5)

In [43]:
# mxnet.gluon.Trainer(params, optimizer, optimizer_params=None, kvstore='device', compression_params=None, update_on_kvstore=None)
# params (ParameterDict) – The set of parameters to optimize.
# optimizer (str or Optimizer) – The optimizer to use.
# optimizer_params (dict) – Key-word arguments to be passed to optimizer constructor. For example, {‘learning_rate’: 0.1}. 
# All optimizers accept learning_rate, wd (weight decay), clip_gradient, and lr_scheduler. 
# See each optimizer’s constructor for a list of additional supported arguments.
# 출처: http://34.201.8.176/versions/io/api/python/gluon/gluon.html?highlight=gluon.trainer#mxnet.gluon.Trainer
trainer = gluon.Trainer(model.collect_params(), 'bertadam',
                        {'learning_rate': lr, 'epsilon': 1e-9, 'wd':0.01})

log_interval = 4
num_epochs = 5

In [44]:
# LayerNorm과 Bias에는 Weight Decay를 적용하지 않는다. 
# ???
for _, v in model.collect_params('.*beta|.*gamma|.*bias').items():
    v.wd_mult = 0.0
params = [
    p for p in model.collect_params().values() if p.grad_req != 'null'
]


In [45]:
def evaluate_accuracy(model, data_iter, ctx=ctx):
    acc = mx.metric.Accuracy()
    i = 0
    for i, (t,v,s, label) in enumerate(data_iter):
        token_ids = t.as_in_context(ctx)
        valid_length = v.as_in_context(ctx)
        segment_ids = s.as_in_context(ctx)
        label = label.as_in_context(ctx)
        output = model(token_ids, segment_ids, valid_length.astype('float32'))
        acc.update(preds=output, labels=label)
        if i > 1000:
            break
        i += 1
    return(acc.get()[1])

In [46]:
#learning rate warmup을 위한 준비 
accumulate = 4
step_size = batch_size * accumulate if accumulate else batch_size # 128
num_train_examples = len(data_train) # 15만개
num_train_steps = int(num_train_examples / step_size * num_epochs) #  5859 = 15만 / 128*5
warmup_ratio = 0.1 # ???
num_warmup_steps = int(num_train_steps * warmup_ratio) # 585 = 5859 *  0.1
step_num = 0


all_model_params = model.collect_params()

In [47]:
# Set grad_req if gradient accumulation is required
if accumulate and accumulate > 1:
    for p in params:
        p.grad_req = 'add'

In [48]:
for epoch_id in range(num_epochs):
    metric.reset()
    step_loss = 0
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(train_dataloader):
        if step_num < num_warmup_steps:
            new_lr = lr * step_num / num_warmup_steps
        else:
            non_warmup_steps = step_num - num_warmup_steps
            offset = non_warmup_steps / (num_train_steps - num_warmup_steps)
            new_lr = lr - offset * lr
        trainer.set_learning_rate(new_lr)
        with mx.autograd.record():
            # load data to GPU
            token_ids = token_ids.as_in_context(ctx)
            valid_length = valid_length.as_in_context(ctx)
            segment_ids = segment_ids.as_in_context(ctx)
            label = label.as_in_context(ctx)

            # forward computation
            out = model(token_ids, segment_ids, valid_length.astype('float32'))
            ls = loss_function(out, label).mean()

        # backward computation
        ls.backward()
        if not accumulate or (batch_id + 1) % accumulate == 0:
          trainer.allreduce_grads()
          nlp.utils.clip_grad_global_norm(params, 1)
          trainer.update(accumulate if accumulate else 1)
          step_num += 1
          if accumulate and accumulate > 1:
              # set grad to zero for gradient accumulation
              all_model_params.zero_grad()

        step_loss += ls.asscalar()
        metric.update([label], [out])
        if (batch_id + 1) % (50) == 0:
            print('[Epoch {} Batch {}/{}] loss={:.4f}, lr={:.10f}, acc={:.3f}'
                         .format(epoch_id + 1, batch_id + 1, len(train_dataloader),
                                 step_loss / log_interval,
                                 trainer.learning_rate, metric.get()[1]))
            step_loss = 0
    test_acc = evaluate_accuracy(model, test_dataloader, ctx)
    print('Test Acc : {}'.format(test_acc))

[Epoch 1 Batch 50/4688] loss=9.1173, lr=0.0000010256, acc=0.516
[Epoch 1 Batch 100/4688] loss=8.6865, lr=0.0000020513, acc=0.521
[Epoch 1 Batch 150/4688] loss=8.4828, lr=0.0000031624, acc=0.538
[Epoch 1 Batch 200/4688] loss=7.5561, lr=0.0000041880, acc=0.581
[Epoch 1 Batch 250/4688] loss=7.1114, lr=0.0000052991, acc=0.609
[Epoch 1 Batch 300/4688] loss=6.7686, lr=0.0000063248, acc=0.631
[Epoch 1 Batch 350/4688] loss=5.6022, lr=0.0000074359, acc=0.656
[Epoch 1 Batch 400/4688] loss=5.4755, lr=0.0000084615, acc=0.675
[Epoch 1 Batch 450/4688] loss=5.3786, lr=0.0000095726, acc=0.688
[Epoch 1 Batch 500/4688] loss=5.0646, lr=0.0000105983, acc=0.700
[Epoch 1 Batch 550/4688] loss=4.7464, lr=0.0000117094, acc=0.712
[Epoch 1 Batch 600/4688] loss=4.6596, lr=0.0000127350, acc=0.723
[Epoch 1 Batch 650/4688] loss=4.4470, lr=0.0000138462, acc=0.732
[Epoch 1 Batch 700/4688] loss=4.5672, lr=0.0000148718, acc=0.740
[Epoch 1 Batch 750/4688] loss=4.6551, lr=0.0000159829, acc=0.746
[Epoch 1 Batch 800/4688] l

KeyboardInterrupt: 